In [1]:
import requests
import json
import os
import time
from datetime import datetime, timedelta

Essa é a requisição para Landing Zone:

In [ ]:
def requisicao(query,one_hour_ago, current_time):   
    # URL da API webhook
    url = "http://192.168.15.18:8000/webhook"
    
    # Dados a serem enviados na requisição
    data = {
        "query": query,
        "from":one_hour_ago,
        "to":current_time
    }

    # Enviando a requisição POST para a API
    response = requests.post(url, json=data)


    # Verificando o status da resposta
    if response.status_code == 200:

        # Geração do arquivo JSON
        webhook_data = response.json()

        if webhook_data["totalResults"] > 0:

            dia_anterior = (datetime.now() - timedelta(days=1) - timedelta(hours=1)).strftime('%Y-%m-%d')

            hm = (datetime.now() - timedelta(hours=1)).strftime('%H-%M')

            folder_path = f'noticias/{dia_anterior}'
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)
            with open(f'{folder_path}/{hm}-news.json', 'w') as f:
                json.dump(webhook_data, f)
            print("Novas notícias foram encontradas e salvas:", webhook_data)

        else:
            print("Não há noticias novas com essa Query")
        
    else:
        # Caso contrário, houve um erro na requisição
        print("Erro ao enviar webhook.")
        if not os.path.exists("./log_erro.txt"):
            with open(f'log_erro.txt', 'w') as f:
                f.write(f"{hm},{response.text}")
        with open(f'news.json', 'a') as f:
            f.write(f"{hm},{response.text}")


Esse laço faz com que seja requisitado de hora em hora o webhook e também transforme todos os jsons do dia anterior em arquivo parquet para o Bronze

In [ ]:
while True:
    query = "dengue"

    dia_anterior = (datetime.now() - timedelta(days=1))
    current_time = dia_anterior.strftime('%Y-%m-%dT%H:%M:%SZ')
    one_hour_ago = (dia_anterior - timedelta(hours=1)).strftime('%Y-%m-%dT%H:%M:%SZ')
    
    # Seleção do tempo de rodagem do código a cada 1h
    if current_time[-6:] == "00:01Z": #é considerado 1 segundo a mais para rodar, pois garante a requisição correta quando for as 00H00
        requisicao(query,one_hour_ago, current_time)
        time.sleep(10) 
    if current_time[-9:] == "00:00:01Z":
        pass
        # codigo de juntar os json do dia em um parquet
        # for arquivo na pasta usando OS:
        #   arquivo = arquivo["article"]
        #   transforma esse arquivo em parquet unico # webhook_data = webhook_data["articles"] # sendo 1 arquivo por dia
        # remover/drop_duplicates nesse arquivo
        # transformar esse arquivo do bronze para o silver (sendo que o silver é um arquivo já existente)

### testes

In [172]:
dia_anterior = (datetime.now() - timedelta(days=1))
current_time = dia_anterior.strftime('%Y-%m-%dT%H:%M:%S')
one_hour_ago = (dia_anterior - timedelta(hours=1)).strftime('%Y-%m-%dT%H:%M:%S')

print(dia_anterior)
print(current_time)
print(one_hour_ago)

2024-03-26 00:13:51.010519
2024-03-26T00:13:51
2024-03-25T23:13:51


In [173]:
query = "dengue"

dia_anterior = (datetime.now() - timedelta(days=1))
current_time = dia_anterior.strftime('%Y-%m-%dT%H:%M:%SZ')
one_hour_ago = (dia_anterior - timedelta(hours=1)).strftime('%Y-%m-%dT%H:%M:%SZ')

# URL da API webhook
url = "http://192.168.15.18:8000/webhook"
    
    # Dados a serem enviados na requisição
data = {
    "query": query,
    "from":"2024-03-25T23:00:01Z",
    "to":"2024-03-26T00:00:01Z"
}

    # Enviando a requisição POST para a API
response = requests.post(url, json=data)


    # Verificando o status da resposta
if response.status_code == 200:

        # Geração do arquivo JSON
    webhook_data = response.json()

    if webhook_data["totalResults"] > 0:
            
        today = datetime.now().strftime('%Y-%m-%d')
        hm = datetime.now().strftime('%H-%M')

        folder_path = f'noticias/{today}'
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        # with open(f'{folder_path}/{hm}-news.json', 'w') as f:
        with open(f'{folder_path}/teste-news.json', 'w') as f: 
            json.dump(webhook_data, f)
        print("Novas notícias foram encontradas e salvas:", webhook_data)

    else:
        print("Não há noticias novas com essa Query")
        
else:
    # Caso contrário, houve um erro na requisição
    print("Erro ao enviar webhook.")




    



Novas notícias foram encontradas e salvas: [{'author': 'The Associated Press', 'content': "SAN JUAN, Puerto Rico -- Puerto Rico's health secretary declared an epidemic on Monday following a spike in dengue cases.\r\nThe U.S. territory of 3.2 million people has reported at least 549 cases so … [+871 chars]", 'description': 'Puerto Rico has declared an epidemic following a spike in dengue cases', 'publishedAt': '2024-03-25T23:02:05Z', 'source': {'id': 'abc-news', 'name': 'ABC News'}, 'title': 'Puerto Rico has declared an epidemic following a spike in dengue cases', 'url': 'https://abcnews.go.com/International/wireStory/puerto-rico-declared-epidemic-spike-dengue-cases-108482841', 'urlToImage': 'https://s.abcnews.com/images/US/abc_news_default_2000x2000_update_16x9_992.jpg'}, {'author': 'Belén Liotti', 'content': '¿Cómo protegerse del mosquito del dengue? 4:52\r\n(CNN Español) -- El secretario de Salud de Puerto Rico, Carlos Mellado, informó este lunes que emitieron una declaración de emer